读完这篇DPO论文后，我的理解：

关于loss函数从Bradley-Terry模型的推导：BT模型在RLHF的场景下，它假设人类更偏好的回答y_w相对于y_l的概率可以表示为p(y_w > y_l) = sigmoid(r(x,y_w) - r(x,y_l))，所以传统RLHF需要先训练一个reward model去拟合这个r(x,y)，然后用RL去最大化这个reward。

而DPO的关键突破就在于，**通过对公式3的RL优化目标求解，得到了最优策略π*的闭式解**：π*(y|x) ∝ π_ref(y|x) exp(r(x,y)/β)。把这个关系代入BT模型，就能直接用策略的概率比来表示偏好概率，从而绕过了显式的reward model。最终的DPO loss就是最大化偏好数据的似然，具体形式是-log σ(β log[π_θ(y_w|x)/π_ref(y_w|x)] - β log[π_θ(y_l|x)/π_ref(y_l|x)])，**这样就把优化reward转化成了直接优化概率分布**。

**关于KL散度的作用**，我认为这是RLHF中一个非常重要的正则化项。如果没有KL约束，模型可能会过度优化reward而产生一些奇怪的输出，比如重复某些能获得高分的模式，或者完全偏离原始语言模型的分布。KL散度KL(π||π_ref)确保了微调后的模型不会偏离预训练模型太远，保持了语言的流畅性和多样性。这其实和PPO中的trust region思想类似，都是为了让优化过程更稳定，避免模型在某个方向上走得太远而崩溃。

**对于RL在LLM上的潜力和应用**，查阅了一些资料，我觉得潜力还是很大的。除了已经很成功的使用DPO、RLHF用于对齐，RL还可以用于代码生成（通过执行结果作为reward）、对话系统（通过用户满意度反馈）、甚至是创造性写作（通过特定风格指标）。关于RL是否能学到人类偏好，基于这篇论文，我认为是可以的；但痛点在于——需要大量标注高质量的偏好数据。DPO本质上是在学习人类标注的偏好分布，标注质量好且覆盖面广，模型才能更好地捕捉到人类的价值观和偏好，缺数据、标数据这也是整个LLM领域的痛点。

**至于RL能否完全替代预训练**，我觉得现阶段不太可能。预训练能让模型学会语言的基本结构和海量知识，而RL很难从零开始学习，需要大量的试错。我觉得RL更适合在已有的强大基础上做精细调整。从头用RL训练需要极其复杂的reward设计，而且样本效率会非常低。想象一下，要让模型仅通过reward信号学会语法、词汇、世界知识，这个探索空间太大太大了。

从论文中的实验结果部分来看，一个有趣的发现是：DPO训练后的模型在分布外的任务上表现也有提升。论文中展示了在summarization任务上训练的模型，在dialogue等其他任务上也有改善，这说明通过偏好学习获得的能力有一定的泛化性，可能是因为模型学到了更通用的"什么是好的回答"的概念。但这种泛化是有限的，对于完全不同领域的输入，效果提升就没那么明显。

总的来说，DPO提供了一种更简洁高效的方式来做偏好对齐，避免了传统RLHF的复杂性。但它还是依赖高质量的偏好数据和强大的预训练。

[Reinforcement Pre-Training](https://arxiv.org/abs/2506.08007)
这一篇文章是第一个RL预训练模型，是RL从后训练角色迈向预训练的一大步，回答以下问题
- 模型的训练数据/语料是什么格式？
- 模型的奖励是如何定义？
- 作者使用了哪些RL方法进行pretrain？了解这些方法并解释其在RLpretrain中的作用和必要性。

### 第一个问题，关于训练数据的格式
训练数据核心来自OmniMATH数据集，这个数据集包含4428个竞赛级数学问题及对应的解答，基础格式是普通文本数据，与传统预训练的文本格式一致。不过，为了让模型聚焦需要推理的内容，而非机械记忆易预测的词，研究中加入了关键的数据过滤步骤：先用Deepseek-R1-Distill-Qwen-1.5B这个小代理模型，计算每个词位置在top-16下一词预测结果上的熵值，再通过设定不同的熵阈值（0.5、1.0、1.5分别对应简单、中等、困难三个难度等级的词），过滤掉低熵的易预测词，只保留高熵的“挑战性词”用于训练，确保训练资源集中在需要推理能力的部分。

### 第二个问题，关于奖励是怎么定义的
模型的奖励设计采用前缀匹配奖励机制，核心是基于字节序列的精确验证，避免了复杂的评分系统，同时能支持多词预测和词汇表外词的验证。具体来说，会先把模型预测的下一词序列（记为y_t^i）和真实文本中该词后续的序列（记为x_≥t）都转换为字节序列，然后判断两个条件是否同时满足：一是预测的字节序列必须是真实字节序列的精确前缀，二是预测字节序列的长度要匹配真实序列中的“有效词边界”——也就是真实续写序列中每个词累积的字节长度集合。只有当这两个条件都满足时，奖励才记为1，否则为0，这种设计能最小化奖励黑客的风险，确保奖励的可验证性。

### 第三个问题，关于用了什么RL方法以及为什么需要它
作者在RPT预训练中采用的RL方法以GRPO算法为核心，同时搭配了多响应采样、动态采样等策略，并且选择Deepseek-R1-Distill-Qwen-14B作为基模型，训练过程中设置零KL惩罚。具体来看，GRPO算法属于策略梯度类算法，能直接优化“最大化期望奖励”这个核心目标，非常适配RPT将下一词预测重构为推理任务的场景，帮助模型逐步学习更优的推理策略；而多响应采样是对每个输入上下文采样8个包含思维链和预测结果的响应，通过多轨迹探索减少单一样本的随机性，更精准地估计奖励期望，避免模型陷入局部最优的低效推理模式；动态采样则是从训练500步开始启用，主要作用是平衡训练效率与效果，防止后期因数据重复导致收敛变慢。
从必要性来看，这些方法的组合是为了适配RPT的核心目标：GRPO确保了策略优化的方向性，多响应采样提升了奖励估计的准确性，动态采样保障了训练的效率，而零KL惩罚则允许模型充分探索更优的推理路径——因为RPT的目标是让模型通过推理自主优化下一词预测，而非被约束在预训练分布内，这种设计能更好地培养模型的推理能力。同时，选择具备基础推理能力的Deepseek-R1-Distill-Qwen-14B作为基模型，也为RL预训练提供了合适的起点，避免模型从零开始学习推理，提升了训练的样本效率。